In [54]:
import pandas as pd

In [55]:
events = pd.read_excel("events.xlsx", engine='openpyxl')
key_segments = pd.read_excel("key segments.xlsx", engine='openpyxl')
segment_data = pd.read_csv("segment_data.csv", sep=";")
monday_speeds = pd.read_csv("monday-speeds.csv", sep=";")
segments_ids_nord_direction = pd.read_csv("north_direction_road_segments.txt", header=None, names=['SegmentId'])

### Event Document

In [56]:
key_segments.loc[key_segments['SegmentID'] == 428389808].head(50)

,RouteID,SegmentID,ValidFrom,ValidTo,Längd,Färdriktn,Latitud,Longitud,Ordning,Väg,Antal kf,Hastighet,Korsning,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
48,1520100002,428389808,2019-04-18,9999-12-31 23:00:00,214,Syd,"61,068809","16,964382",35,E4,1kf,100,Påfart,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
key_segments.loc[key_segments['Färdriktn'] != "Syd"].head(50)

,RouteID,SegmentID,ValidFrom,ValidTo,Längd,Färdriktn,Latitud,Longitud,Ordning,Väg,Antal kf,Hastighet,Korsning,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
97,1520100060,1071754019,2019-04-20 01:00:00,9999-12-31 23:00:00,927,Norr,"61,1431","16,901216",10,83,1kf,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,1520100070,1071754180,2019-04-20 02:00:00,9999-12-31 23:00:00,934,Norr,"61,161885","16,866368",13,83,1kf,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99,1520100071,1071754151,2019-04-20 03:00:00,9999-12-31 23:00:00,930,Norr,"61,165335","16,851285",14,83,1kf,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,1520100072,1071754251,2019-04-20 04:00:00,9999-12-31 23:00:00,932,Norr,"61,149015","16,889462",11,83,1kf,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101,1520100073,375118964,2019-04-20 05:00:00,9999-12-31 23:00:00,931,Norr,"61,137057","16,913236",9,83,1kf,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102,1520100074,428410138,2019-04-20 06:00:00,9999-12-31 23:00:00,932,Norr,"61,13098","16,924403",8,83,1kf,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103,1520100075,1071754109,2019-04-20 07:00:00,9999-12-31 23:00:00,929,Norr,"61,124236","16,93289",7,83,1kf,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104,1520100076,1071754436,2019-04-20 08:00:00,9999-12-31 23:00:00,808,Norr,"61,107871","16,939029",5,83,1kf,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105,1520100077,1071754407,2019-04-20 09:00:00,9999-12-31 23:00:00,929,Norr,"61,115522","16,934902",6,83,1kf,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106,1520100078,1071754470,2019-04-20 10:00:00,9999-12-31 23:00:00,1048,Norr,"61,100945","16,947959",4,83,1kf,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### SegmentID direction and coordinates

In [58]:
key_segments.head(50)

,RouteID,SegmentID,ValidFrom,ValidTo,Längd,Färdriktn,Latitud,Longitud,Ordning,Väg,Antal kf,Hastighet,Korsning,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,1520100057,1071754891,2019-04-16 00:00:00,9999-12-31 00:00:00,930,Syd,"61,165335","16,851284",1,83,1kf,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1520100058,1071754134,2019-04-16 01:00:00,9999-12-31 01:00:00,934,Syd,"61,161885","16,866368",2,83,1kf,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1520100059,1071754197,2019-04-16 02:00:00,9999-12-31 02:00:00,932,Syd,"61,149015","16,889462",4,83,1kf,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1520100061,1071754002,2019-04-16 03:00:00,9999-12-31 03:00:00,932,Syd,"61,137055","16,91324",6,83,1kf,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1520100062,428397715,2019-04-16 04:00:00,9999-12-31 04:00:00,938,Syd,"61,131006","16,924362",7,83,1kf,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1520100063,1071754066,2019-04-16 05:00:00,9999-12-31 05:00:00,929,Syd,"61,124233","16,932893",8,83,1kf,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1520100064,374986318,2019-04-16 06:00:00,9999-12-31 06:00:00,929,Syd,"61,115489","16,9349",9,83,1kf,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1520100065,1071754391,2019-04-16 07:00:00,9999-12-31 07:00:00,935,Syd,"61,107371","16,939636",10,83,1kf,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1520100066,1071754420,2019-04-16 08:00:00,9999-12-31 08:00:00,915,Syd,"61,100528","16,948509",11,83,1kf,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1520100067,1071754453,2019-04-16 09:00:00,9999-12-31 09:00:00,938,Syd,"61,092998","16,954625",12,83,1kf,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### SegmentId meta data

In [59]:
segment_data.head(1)

,SegmentId,Length,Road,Antal kf,Max_speed
0,1071754891,930,83,1kf,90


### SegmentID and speeds

In [60]:
monday_speeds.head(1)

,SegmentId,StartTime,EndTime,Speed
0,1071707139,2019-06-03 00:00:00,2019-06-03 00:30:00,NaN


# The Goal

- Create a deep learning model to predict traffic speed after a traffic event until it returns to normal speed

# What variables needed to do predictions?
- SegmentID
- Speed in time
- Events
    - Code/Text
    - CREATION_TIME 	
    - COMPLETION_TIME 	
    - START_TIME 	
    - END_TIME

- Weather data is not needed to do the project and therefore not included here. Most important is to link speed and events first and then we can expand with weather data

# Observations

#### Event Document
- The event code
- X and Y location (this is most likely not latitude and longitude)

#### SegmentID and Speed
- segmentID
- Speed

#### SegmentId and coordinates
- SegmentID
- Latitude and Longitude coordinates

# My take on it

It's unclear if Latitude and Longitude can be connected to the X,Y from the event document which is required to link SegmentID, Speed and Events.

<br>

Therefore, questions to answer
- What type of coordinate do the X,Y represent in the event document since they don't look like latitude and longitude?

In [61]:
df = pd.read_csv("../data/events-with-lat-long-segmentID.csv", sep=",")

In [62]:
this_segment = df[df['SegmentID'] == 226255131]

In [63]:
this_segment

,Unnamed: 0,Skapad_År,Skapad_Månad,NAME,EVENT_CODE,EVENT_TEXT,Location_text,Väg,Riktning,LANE_MASK,CREATION_TIME,COMPLETION_TIME,START_TIME,END_TIME,Note,X,Y,latitude,longitude,SegmentID
1338,1338,2021,Juli,2021-07-27/23:16:50-1/4,40,Vägarbete,E4 från Axmartavlan (202) till Trafikplats Tön...,E4,Norr,OOOOOOOOO|XOOOOOOOO,2021-07-27 23:16:50,2021-07-28 00:32:15,2021-07-27 23:16:50,2021-07-28 00:30:00,Underhållsarbete i höjd med Högbacka. Risk för...,6765123,607357,61.00636,16.985545,226255131


In [64]:
segment_ids = []
segment_ids_count = []

#temp = df.loc[(df['EVENT_CODE'] != 40) & df['EVENT_CODE'] != 2)] 

# Only take south(syd) direction into consideration because 428389808 segment should only be for south dir.
temp = df.loc[(df['EVENT_CODE'] != 40) & 
              (df['EVENT_CODE'] != 2) &
              (df['Riktning'] == "Syd")] 


for segment_id in temp['SegmentID'].values:
    
    
    counts = temp['SegmentID'].value_counts()
    
    #print(counts)
    
    if segment_id not in segment_ids:
        
        
        count = counts.get(segment_id)
        
        print(segment_id, count)
        
        segment_ids_count = [str(segment_id), count]
        segment_ids.append(segment_id)
        

1071863723 2
428403354 62
428389808 152
1071723711 60
1071723605 2
1071707139 2
1071723800 2
1071707171 4
226992808 1
226944388 4
1071727508 4
428403369 12
1071723686 4
1071703593 2
428402118 2
1071863848 1
1071723815 2
428403366 2
1071845973 2
1071754891 2
1071845832 3
1071863801 2
1071891776 2
1071832566 3
1071703625 3
428390707 2
1071863817 1
428389804 1
1071863641 1
428403355 2
1071727530 1
1071790234 1
1071846005 1
1071707092 3
1071832389 1
1071727490 2
1071707154 1
1071723854 2
1071845814 2
428403352 2
1071832355 2
1071707121 1
374870394 1
1071846212 1
375160804 1
226910815 1
375114290 1
428403356 3
1071846041 1
1071727524 1
226736042 2
1071832374 1
428403367 1
1071845955 2
428390706 1
428390709 1
428397753 1
1071863659 1
1071703610 1
1071832301 1
1071703530 2
428403357 1
1071845863 1


We can see that segment_id: 428389808 has the most events

In [65]:
temp = temp[temp['SegmentID'] == 428389808]

In [66]:
temp.head(100)

,Unnamed: 0,Skapad_År,Skapad_Månad,NAME,EVENT_CODE,EVENT_TEXT,Location_text,Väg,Riktning,LANE_MASK,CREATION_TIME,COMPLETION_TIME,START_TIME,END_TIME,Note,X,Y,latitude,longitude,SegmentID
11,11,2019,Januari,2019-01-05/17:07:51-1/4,32,Stillastående fordon,E4 från Trafikplats Tönnebro (203) till Axmart...,E4,Syd,OOOOOOOOX|OOOOOOOOO,2019-01-05 17:07:50,2019-01-05 17:30:31,2019-01-05 17:07:51,2019-01-05 17:30:00,Personbil med haveri,6771941,606024,61.067895,16.964691,428389808
21,21,2019,Januari,2019-01-08/22:35:04-1/4,255,Bärgning,E4 från Trafikplats Tönnebro (203) till Axmart...,E4,Syd,OOOOOOOOX|OOOOOOOOO,2019-01-08 22:35:04,2019-01-08 23:07:08,2019-01-08 22:35:04,2019-01-08 23:05:00,Personbil som ska bärgas mellan Tönnebro och A...,6771941,606024,61.067895,16.964691,428389808
34,34,2019,Januari,2019-01-17/01:09:02-1/4,255,Bärgning,E4 från Trafikplats Tönnebro (203) till Axmart...,E4,Syd,XXXXXXXXX|OOOOOOOOO,2019-01-17 01:09:02,2019-01-17 02:09:53,2019-01-17 01:09:02,2019-01-17 02:15:00,Bärgning av personbil strax söder om Högbacka ...,6771941,606024,61.067895,16.964691,428389808
36,36,2019,Januari,2019-01-17/14:18:15-1/4,255,Bärgning,E4 från Trafikplats Tönnebro (203) till Axmart...,E4,Syd,XXXXXXXXX|OOOOOOOOO,2019-01-17 14:18:14,2019-01-17 14:32:57,2019-01-17 14:18:15,2019-01-17 14:30:00,Bärgning pågår. Vägen stängd ca Kl. 14:30,6771941,606024,61.067895,16.964691,428389808
39,39,2019,Januari,2019-01-18/12:28:50-1/4,255,Bärgning,E4 från Trafikplats Tönnebro (203) till Axmart...,E4,Syd,XXXXXXXXX|OOOOOOOOO,2019-01-18 12:28:50,2019-01-18 13:03:49,2019-01-18 12:28:50,2019-01-18 13:00:00,bärgning av personbil södergående,6771941,606024,61.067895,16.964691,428389808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
836,836,2020,Juli,2020-07-26/18:42:48-1/4,255,Bärgning,E4 från Trafikplats Tönnebro (203) till Axmart...,E4,Syd,OOOOOOOOX|OOOOOOOOO,2020-07-26 18:42:48,2020-07-26 19:08:47,2020-07-26 18:42:48,2020-07-26 19:30:00,"Bärgning av en personbil i höjd med Högbacka,",6771941,606024,61.067895,16.964691,428389808
853,853,2020,Augusti,2020-08-02/15:37:16-1/4,169,Långsam kö,E4 från Trafikplats Tönnebro (203) till Trafik...,E4,Syd,OOOOOOOOX|OOOOOOOOO,2020-08-02 15:37:15,2020-08-02 20:45:04,2020-08-02 15:37:16,2020-08-02 21:00:00,Tät trafik Tönnebro - Gävle i södergående rikt...,6771941,606024,61.067895,16.964691,428389808
876,876,2020,Augusti,2020-08-13/03:32:28-1/4,32,Stillastående fordon,E4 från Trafikplats Tönnebro (203) till Axmart...,E4,Syd,OOOOOOOOX|OOOOOOOOO,2020-08-13 03:32:27,2020-08-13 04:08:21,2020-08-13 03:32:28,2020-08-13 04:30:00,Lastbil med punktering strax söder om Tönnebro...,6771941,606024,61.067895,16.964691,428389808
896,896,2020,Augusti,2020-08-22/20:33:44-1/4,257,Rastplats #,E4 från Trafikplats Tönnebro (203) till Axmart...,E4,Syd,OOOOOOOOO|OOOOOOOOO,2020-08-22 20:33:44,2020-08-24 07:45:56,2020-08-22 20:33:44,2020-08-24 08:00:00,"E 4 Högbacka rastplats, i riktning mot Gävle.\...",6771941,606024,61.067895,16.964691,428389808


In [67]:
temp.to_csv("RoadEvents_SegmentID_428389808.csv", sep=",")